In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

In [2]:
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
'''# Read the file data and specifie delimiter'''
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
    
'''# For each line in the file we will We remove newlines chars for nicer display'''
text = text.replace("\n", " ") 
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

'''now we will cut text into characters without repetition'''
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

600901/600901 [==============================] - 30s 50us/step
Corpus length: 600893
Total chars: 56
Number of sequences: 200285


C:\Users\Zaid's Laptop\AppData\Local\Temp\ipykernel_25652\1178333198.py:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\Zaid's Laptop\AppData\Local\Temp\ipykernel_25652\1178333198.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)

In [3]:
model = keras.Sequential(
    [   # Define an input sequence and process it.
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
'''
Before fitting our model, we choosed softmax as activation function for output layer and rmsprop with 0.01 learning rate as optimizer .
'''
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

## Prepare the text sampling function

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [5]:
'''Now we will fit our model, and for each step (epoch) we will print the generated text by our model'''
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

1565/1565 [==============================] - 87s 54ms/step - loss: 1.9231

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "ctions? in the end, however, though sile"
...Generated:   the stall and a propers and in the stand and and and and the streat and and stand the moral and denered the stand and stand the religion the stall and the perhaps and and so the propers and and and the still the still the streat the still the still the stear the still the streat the still the streat the for the self-propersed and devened the stand and and the still the self-probatist and and and 

...Diversity: 0.5
...Generating with seed: "ctions? in the end, however, though sile"
...Generated:   an in a moral in in the self-pleces the age experience of the happined by the stander that is and deverence, from the self-pleder of must and will to the self-probease of the in larking and the can condition that the sthiles in the recestation the heartion the self-perhaps and a properian

1565/1565 [==============================] - 85s 55ms/step - loss: 1.4027

Generating text after epoch: 4
...Diversity: 0.2
...Generating with seed: " indeed, every instance of taking a thin"
...Generated:  g the most strength, and the sense of the community of the extent the same man who was a sense of the senses of the seems to the personal the extent to be interesting the most such a commander the same sould of the such a so the end the such a south of the most such a such a such a south of the personal the enduring the established to the same soul of the enduring the same moral the most strength 

...Diversity: 0.5
...Generating with seed: " indeed, every instance of taking a thin"
...Generated:  g and really everything of the past of the subling attiods of commander the more incertance to the heart the warp the such a soltation of the master scholar spelism, and others in the backmen that the have a south of the sensible to the expeasing into all that it is not not not the compreh

1565/1565 [==============================] - 95s 61ms/step - loss: 1.3383

Generating text after epoch: 8
...Diversity: 0.2
...Generating with seed: "itchen--the development of mankind has b"
...Generated:  een the true special superiority and present, the same conscience of the same are the such a such a such a such a such a such a such a more and the same spirit and the same things and the decodes of the intellect and intellect of the same as the transists of the same profound the such a such a such a such a more than the same spirit of the subject of the same development of the sacrifice of the sa

...Diversity: 0.5
...Generating with seed: "itchen--the development of mankind has b"
...Generated:  een intercalled he can he would be the speaks there is not the present of the sadularily consequence in the free spirit the old something the nature than the more to its beamus in our part of a can the more impresent to them--and the artists of conditional permithing and as the religious a

C:\Users\Zaid's Laptop\AppData\Local\Temp\ipykernel_25652\3050522900.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  ne may as the strength of the probably in order to the soul of the contrasted the probably as the respect to the probably and the same things of the freedom in the soul of the former than a soul of the conscience to the worst of the soul as a south and the strength of the consisted the consisted and experience of the consisted and soul of the soul of the highest soul and the consequence of the sam

...Diversity: 0.5
...Generating with seed: "in the sternness of its principle that o"
...Generated:  ne says reach of nature and such a honation of their new existence of the soul of the most the bearm and striving the struct of souls of the good as in the reason of the part, the such a stronger in share as the present preval such a language to the things in the to his day, in as a speak of a worst of its say.                                                                                        

...Diversity: 1.0
...Generating with seed: "in the sternness of its principle th

...

1565/1565 [==============================] - 340s 217ms/step - loss: 1.2546

Generating text after epoch: 23
...Diversity: 0.2
...Generating with seed: "he acting individual as regards himself,"
...Generated:   and the self-desire the most and the same time and interpretation of the sense of the sense of the process of the procession and and strongest and interpretation of the profound to the state of the sense of the strange in the such a philosophers and the consist and and stronger in such and the standard itself and and the problem of the sense of the same time and profound to the self-desire and st

...Diversity: 0.5
...Generating with seed: "he acting individual as regards himself,"
...Generated:   it is not almost recultation of its processius and the an extensive and probable of the belief the such disguised to the misunder to the freedom, in the one because the reals sought to his own spirit and and the day, and in which the state the self-said to be must be the process and ex

1565/1565 [==============================] - 292s 186ms/step - loss: 1.2427

Generating text after epoch: 27
...Diversity: 0.2
...Generating with seed: "nsipid, traffic-virtuous people to whom "
...Generated:  is the sentiment of the sense of the sense of the suress and suress of the sentiment, and comparison the sense of the sense of the probability of the same compression, and also the suress of the most and surely and as the sense, and all the strength of the suress of the sense of the suress of the sense of the religious and and probably and sense of the sured and sure--and the sentiment of the prob

...Diversity: 0.5
...Generating with seed: "nsipid, traffic-virtuous people to whom "
...Generated:  in a sense--the most personality, not in short, the conscience of such a will and person the most which its deception of a respectable means and and seen one of the same backness, and more connection of all the problem the most also a spectatorive and the probax art of the same life and

1565/1565 [==============================] - 313s 200ms/step - loss: 1.2343

Generating text after epoch: 31
...Diversity: 0.2
...Generating with seed: "most extensive (perhaps disturbing and d"
...Generated:  esires and woman is the self-destration of the strength of the world of the person and well to an action that the deceptive morality and the self-desires of the self-dective and the moral conscience and intellectual conscience and the individual conscience that it is all the morality and society and and the soul of the souls of the self-dective that is all the self-desire and self-desire and consc

...Diversity: 0.5
...Generating with seed: "most extensive (perhaps disturbing and d"
...Generated:  omain of men and such as a strong the standard and with the self-duling with the self-dective in the philosophy, the constantly and man in a deceptive penited than the philosophy, as the person of the whole circumstances and order to him the world of the world of all the world and condi

1565/1565 [==============================] - 384s 245ms/step - loss: 1.2253

Generating text after epoch: 35
...Diversity: 0.2
...Generating with seed: "gon is: psychological observation--is am"
...Generated:  ong the sensible conscience of the stronger of the sension and the sensible and sublime of the thoughts of the conscience and profound the sortions and profound thought of the sublime and concerning the sortions and sense of the soul of the superficial to the same that is a probably of the sensible of the superficial and conception of the thought we has been way the proper proper the sensible and 

...Diversity: 0.5
...Generating with seed: "gon is: psychological observation--is am"
...Generated:  ong the constraint the slight of be for a hunging profound, perhaps as he for the standard of the suffering of the christian of the former to false and world.   30  =the proper the christian consciousness, and indeed the most interpretation, the sortions and a serious from a signs of su

1565/1565 [==============================] - 466s 298ms/step - loss: 1.2173

Generating text after epoch: 39
...Diversity: 0.2
...Generating with seed: " in a manner different from what would b"
...Generated:  e in the superior to the constant and and and interested the strength, and which we may be it the conscious of the expedined to the sense of the most constrain and the most and conscious of the philosopher the superior and as the proceed in the sense and modern destined and and superior the sense of the sense of the sense of the strange--the probably in the sense of the most and superior that the 

...Diversity: 0.5
...Generating with seed: " in a manner different from what would b"
...Generated:  e strong fine the condition would not is not and live as constant of the life and the world contrary of man who are so that one has to destruct to mean and former character of the experiant of the spirit despect of capacity and probably and end by the desire and deserve ow to the but th

In this exemple we build a LSTM model to generate text character-by-character. We h've splited the texts into characters and build a single layer LSTM model with softmax activation function. from epoch 20 we can recognize that the text become more coherent.